In [1]:
import pandas as pd
import numpy as np
import os 
import re
import matplotlib.pyplot as plt
import scipy
# from scipy.stats.stats import pearsonr 
import os
from pathlib import Path
import math
from statsmodels.stats.multitest import multipletests
import statsmodels
from scipy.stats import ttest_rel, ttest_ind
from scipy import stats

In [2]:
normal =pd.read_csv('data/lusc-rsem-fpkm-tcga_paired.txt', sep="\t", header=0)
print(len(normal))
print(normal.columns)
print(normal.head(3))

19648
Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'TCGA-43-7657', 'TCGA-58-8386',
       'TCGA-22-5478', 'TCGA-22-5472', 'TCGA-43-5670', 'TCGA-60-2709',
       'TCGA-22-5489', 'TCGA-77-8007', 'TCGA-22-5471', 'TCGA-22-4609',
       'TCGA-22-5482', 'TCGA-56-8082', 'TCGA-22-5483', 'TCGA-56-8623',
       'TCGA-33-4587', 'TCGA-56-7579', 'TCGA-43-3394', 'TCGA-34-8454',
       'TCGA-77-7338', 'TCGA-43-6143', 'TCGA-43-6773', 'TCGA-51-4080',
       'TCGA-34-7107', 'TCGA-39-5040', 'TCGA-43-6771', 'TCGA-92-7340',
       'TCGA-77-7138', 'TCGA-77-7142', 'TCGA-56-7823', 'TCGA-22-5491',
       'TCGA-22-5481', 'TCGA-90-7767', 'TCGA-77-8008', 'TCGA-56-7730',
       'TCGA-33-6737', 'TCGA-56-7582', 'TCGA-56-7731', 'TCGA-77-7337',
       'TCGA-43-7658', 'TCGA-56-8201', 'TCGA-85-7710', 'TCGA-56-7580',
       'TCGA-43-6647', 'TCGA-90-6837', 'TCGA-56-8083', 'TCGA-51-4079',
       'TCGA-56-7222', 'TCGA-22-4593', 'TCGA-51-4081', 'TCGA-56-8309'],
      dtype='object')
  Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TC

In [3]:
cancer = pd.read_csv('data/lusc-rsem-fpkm-tcga-t_paired.txt', sep="\t", header=0)
print(len(cancer))
print(cancer.columns)
print(cancer.head(3))

19648
Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'TCGA-43-7657', 'TCGA-58-8386',
       'TCGA-22-5478', 'TCGA-22-5472', 'TCGA-43-5670', 'TCGA-60-2709',
       'TCGA-22-5489', 'TCGA-77-8007', 'TCGA-22-5471', 'TCGA-22-4609',
       'TCGA-22-5482', 'TCGA-56-8082', 'TCGA-22-5483', 'TCGA-56-8623',
       'TCGA-33-4587', 'TCGA-56-7579', 'TCGA-43-3394', 'TCGA-34-8454',
       'TCGA-77-7338', 'TCGA-43-6143', 'TCGA-43-6773', 'TCGA-51-4080',
       'TCGA-34-7107', 'TCGA-39-5040', 'TCGA-43-6771', 'TCGA-92-7340',
       'TCGA-77-7138', 'TCGA-77-7142', 'TCGA-56-7823', 'TCGA-22-5491',
       'TCGA-22-5481', 'TCGA-90-7767', 'TCGA-77-8008', 'TCGA-56-7730',
       'TCGA-33-6737', 'TCGA-56-7582', 'TCGA-56-7731', 'TCGA-77-7337',
       'TCGA-43-7658', 'TCGA-56-8201', 'TCGA-85-7710', 'TCGA-56-7580',
       'TCGA-43-6647', 'TCGA-90-6837', 'TCGA-56-8083', 'TCGA-51-4079',
       'TCGA-56-7222', 'TCGA-22-4593', 'TCGA-51-4081', 'TCGA-56-8309'],
      dtype='object')
  Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TC

In [4]:
def check(normal, cancer):
    if not (normal.columns == cancer.columns).all():
        print('Different Columns')
    if not (len(normal) == len(cancer)):
        print('Different length')
    else: 
        print('checked')
# print(normal.columns == cancer.columns)
# print(len(normal) == len(cancer))
check(normal, cancer)

checked


In [5]:
print((normal.columns == cancer.columns).all())

True


In [6]:
print(len(normal.columns))

52


In [7]:
normal_x = normal[((normal == 0).astype(int).sum(axis=1))<25]
cancer_x = cancer[((cancer == 0).astype(int).sum(axis=1))<25]
print(len(normal_x), len(cancer_x))

17626 17717


In [8]:
names = np.intersect1d(normal_x.Hugo_Symbol, cancer_x.Hugo_Symbol)

normal_x = normal_x[normal_x['Hugo_Symbol'].isin(names)]
cancer_x = cancer_x[cancer_x['Hugo_Symbol'].isin(names)]
normal_x.dropna()
cancer_x.dropna()
print(len(normal_x), len(cancer_x))

17275 17275


In [9]:
check(normal_x, cancer_x)

checked


In [10]:
np.array(normal_x.iloc[0][2:])

array([62.12, 130.6, 33.06, 35.5, 73.03, 60.39, 92.05, 66.65, 54.33,
       15.56, 55.49, 30.34, 14.45, 100.83, 131.51, 59.55, 87.03, 71.5,
       23.08, 48.87, 72.52, 115.97, 45.21, 74.58, 8.92, 27.44, 40.64,
       35.76, 59.13, 20.86, 37.05, 48.18, 51.71, 49.56, 72.01, 21.94,
       27.44, 35.0, 76.17, 28.86, 90.77, 59.55, 40.07, 22.92, 29.91,
       82.29, 4.7, 37.32, 43.63, 77.25], dtype=object)

# Normality test 

## paired 

In [25]:
df=normal_x.drop("Hugo_Symbol",axis=1).drop("Entrez_Gene_Id",axis=1)-cancer_x.drop("Hugo_Symbol",axis=1).drop("Entrez_Gene_Id",axis=1)
for i in range (len(df)):
        sample = np.array(df.iloc[i][2:-1])
        stat,pval = stats.shapiro(sample)
        if pval > 0.05 :
            df.at[i,'paired'] = 'ND'
        else: 
            df.at[i,'paired'] = 'NOT'
norm = len(df[df['paired'] == 'ND'])
Not = len(df[df['paired'] == 'NOT'])
print(f"no of normally distributed {norm} and no of not normally distributed {Not}")

TypeError: unsupported operand type(s) for -: 'str' and 'str'

### Independent 

In [11]:
dfs = [normal_x, cancer_x]
for df in dfs:
    df['normality'] = ''
    for i in range (len(df)):
        sample = np.array(df.iloc[i][2:-1])
        stat,pval = stats.shapiro(sample)
        if pval > 0.05 :
            df.at[i,'normality'] = 'ND'
        else: 
            df.at[i,'normality'] = 'NOT'
    df.head(3)


In [12]:
counter = 0
x = ['normal', 'cancer']
for df in dfs:
    norm = len(df[df['normality'] == 'ND'])
    not_norm = len(df[df['normality'] == 'NOT'])
    print('dataframe {} has {} normally distrbuted genes and {} not normally distributed'.format( x[counter],norm, not_norm))
    counter +=1

dataframe normal has 6994 normally distrbuted genes and 10281 not normally distributed
dataframe cancer has 2028 normally distrbuted genes and 15247 not normally distributed


In [ ]:
data = [normal_x.dropna().copy(), cancer_x.dropna().copy()]
for df in data:
    df.drop('Hugo_Symbol',axis=1, inplace=True)
    df.drop('Entrez_Gene_Id',axis=1, inplace=True)
    df.drop('normality',axis=1, inplace=True)
    print(df.columns)

In [21]:
data[0]

,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,TCGA-22-5471,TCGA-22-4609,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,54.33,15.56,...,90.77,59.55,40.07,22.92,29.91,82.29,4.70,37.32,43.63,77.25
1,185.11,283.05,119.26,169.07,165.57,161.02,131.51,198.47,175.07,147.06,...,185.11,119.26,102.97,123.50,264.03,194.36,166.73,105.15,185.11,356.05
2,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,603.67,555.41,...,813.63,2400.97,543.96,2193.99,540.19,521.76,253.23,764.36,518.15,878.17
3,41.81,18.29,40.93,67.12,54.72,29.27,20.26,23.76,28.04,39.22,...,34.51,70.01,57.49,57.89,67.12,34.51,22.10,31.90,28.24,49.91
4,133.36,214.27,108.14,109.66,190.34,211.31,96.01,208.38,120.10,239.52,...,251.48,209.84,120.10,109.66,155.50,162.14,277.20,86.43,164.42,155.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19641,215.77,148.09,83.45,232.94,167.90,80.01,161.02,104.42,81.14,113.56,...,145.02,202.66,111.99,371.22,154.42,289.02,32.59,191.67,204.07,88.26
19642,6792.79,5441.30,5831.91,5329.30,5711.87,6164.49,8134.41,6516.03,8598.28,5633.22,...,3955.48,4869.99,6935.54,4836.35,7230.10,6250.56,6840.04,5292.48,3928.15,7130.55
19643,423.61,529.06,660.68,620.67,518.15,848.22,366.09,1073.91,363.56,366.09,...,559.28,579.04,420.68,366.09,334.46,258.57,329.84,1030.12,665.29,633.73
19646,0.00,0.00,0.99,3.59,2.23,3.35,0.00,7.34,1.41,0.00,...,6.94,1.00,0.00,4.58,0.96,3.06,1.83,1.77,3.63,2.56


# As the data is not normally distributed ,Only 1124 are normally distributed,

## on the paired will used wilcoxon

In [ ]:
stat, p = stats.wilcoxon(data[0], data[1], axis=1, alternative='less')
wilcoxon = pd.DataFrame(list(zip(normal_x.dropna().Hugo_Symbol, normal_x.dropna().Entrez_Gene_Id, normal_x.dropna().normality, p, stat)))
wilcoxon.columns =['Hugo_Symbol', 'Entrez_Gene_Id', 'normality', 'p', 'stat']
wilcoxon['expression'] = np.where(wilcoxon['p']>0.05, 'not', 'expressed')
print(len(wilcoxon))
wilcoxon.head(10)

### on the whole independant dataset we'll use mann

In [24]:
stat, p =  stats.mannwhitneyu((data[0], data[1], axis=1, alternative='less')
mannwhitneyu = pd.DataFrame(list(zip(normal_x.dropna().Hugo_Symbol, normal_x.dropna().Entrez_Gene_Id, normal_x.dropna().normality, p, stat)))
mannwhitneyu.columns =['Hugo_Symbol', 'Entrez_Gene_Id', 'normality', 'p', 'stat']
mannwhitneyu['expression'] = np.where(mannwhitneyu['p']>0.05, 'not', 'expressed')
print(len(mannwhitneyu))
mannwhitneyu.head(10)

TypeError: wilcoxon() got an unexpected keyword argument 'axis'

In [ ]:
mannwhitneyu.to_csv("indep.csv")
wilcoxon.to_csv("paired.csv")